# Q1 : Review existing unstructured data and transform as needed

### Import Libraries

In [1]:
import pandas as pd

# Brands Data

In [2]:
# The actual Json file path is defined
file_path = 'C:/Users/ASUS/Downloads/Fetch/brands.json'

# Load the JSON file into a DataFrame
brands = pd.read_json(file_path, lines=True)

# Display the DataFrame
print(brands)


                                       _id       barcode            category  \
0     {'$oid': '601ac115be37ce2ead437551'}  511111019862              Baking   
1     {'$oid': '601c5460be37ce2ead43755f'}  511111519928           Beverages   
2     {'$oid': '601ac142be37ce2ead43755d'}  511111819905              Baking   
3     {'$oid': '601ac142be37ce2ead43755a'}  511111519874              Baking   
4     {'$oid': '601ac142be37ce2ead43755e'}  511111319917      Candy & Sweets   
...                                    ...           ...                 ...   
1162  {'$oid': '5f77274dbe37ce6b592e90c0'}  511111116752              Baking   
1163  {'$oid': '5dc1fca91dda2c0ad7da64ae'}  511111706328  Breakfast & Cereal   
1164  {'$oid': '5f494c6e04db711dd8fe87e7'}  511111416173      Candy & Sweets   
1165  {'$oid': '5a021611e4b00efe02b02a57'}  511111400608             Grocery   
1166  {'$oid': '6026d757be37ce6369301468'}  511111019930              Baking   

          categoryCode                 

In [3]:
# Flatten the '_id' column as it is a dictionary containing an Object ID ($oid). 
#This is typical in data sourced from systems like MongoDB, where each document has a unique identifier stored in this format.
brands['_id'] = brands['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else x)


# 'cpg' column is a nested column in the dataframe, hence it needs to be normalized to split those values

if 'cpg' in brands.columns:
    cpg_normalized = pd.json_normalize(brands['cpg'])

    # Add a prefix to all column names in the normalized DataFrame
    cpg_normalized = cpg_normalized.add_prefix('cpg_')

    # Drop the original 'cpg' column from brands and concatenate the normalized data
    brands_final = pd.concat([brands.drop('cpg', axis=1), cpg_normalized], axis=1)

brands_final

,_id,barcode,category,categoryCode,name,topBrand,brandCode,cpg_$ref,cpg_$id.$oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...,...,...,...,...,...,...,...
1162,5f77274dbe37ce6b592e90c0,511111116752,Baking,BAKING,test brand @1601644365844,NaN,NaN,Cogs,5f77274dbe37ce6b592e90bf
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NaN,DIPPIN DOTS CEREAL,Cogs,53e10d6368abd3c7065097cc
1164,5f494c6e04db711dd8fe87e7,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,TEST BRANDCODE @1598639215217,Cogs,5332fa12e4b03c9a25efd1e7
1165,5a021611e4b00efe02b02a57,511111400608,Grocery,NaN,LIPTON TEA Leaves,0.0,LIPTON TEA Leaves,Cogs,5332f5f6e4b03c9a25efd0b4


### Missing Values

In [4]:
# Display the number of missing values in each column
print(brands_final.isnull().sum())

_id               0
barcode           0
category        155
categoryCode    650
name              0
topBrand        612
brandCode       234
cpg_$ref          0
cpg_$id.$oid      0
dtype: int64


There are significant amount of missing data and has to be treated. Category, category code, brand code can be replaced with mode or unknown. topBrand can be replaced with 0

### Duplicate Rows

In [5]:
# Check for duplicate rows
duplicate_rows = brands_final.duplicated()
print(f"Number of duplicate rows: {duplicate_rows.sum()}")

Number of duplicate rows: 0


There are no duplicate rows hence each entry in the dataset is unique

### Nested Data Uniformity

In [6]:
# Assuming `cpg` should contain an '$oid' for each entry
brands['cpg'].apply(lambda x: isinstance(x, dict) and '$oid' in x['$id'] if isinstance(x, dict) and '$id' in x else False).value_counts()

True    1167
Name: cpg, dtype: int64

DataFrame correctly contains a dictionary with an id key, and each id has oid. This consistency in the structure indicates good data integrity for the cpg column, showing that all 1167 entries adhere to the expected format.

# Receipts Data

In [7]:
# The actual Json file path is defined
file_path = 'C:/Users/ASUS/Downloads/Fetch/receipts.json'

# Load the JSON file into a DataFrame
receipts = pd.read_json(file_path, lines=True)

# Display the DataFrame
print(receipts)

                                       _id  bonusPointsEarned  \
0     {'$oid': '5ff1e1eb0a720f0523000575'}              500.0   
1     {'$oid': '5ff1e1bb0a720f052300056b'}              150.0   
2     {'$oid': '5ff1e1f10a720f052300057a'}                5.0   
3     {'$oid': '5ff1e1ee0a7214ada100056f'}                5.0   
4     {'$oid': '5ff1e1d20a7214ada1000561'}                5.0   
...                                    ...                ...   
1114  {'$oid': '603cc0630a720fde100003e6'}               25.0   
1115  {'$oid': '603d0b710a720fde1000042a'}                NaN   
1116  {'$oid': '603cf5290a720fde10000413'}                NaN   
1117  {'$oid': '603ce7100a7217c72c000405'}               25.0   
1118  {'$oid': '603c4fea0a7217c72c000389'}                NaN   

                                bonusPointsEarnedReason  \
0     Receipt number 2 completed, bonus point schedu...   
1     Receipt number 5 completed, bonus point schedu...   
2                            All-receipts 

In [8]:
# Flatten the '_id' column
receipts['_id'] = receipts['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else x)

# Function to convert nested date fields from Unix timestamp to datetime
def convert_dates(data):
    if isinstance(data, dict) and '$date' in data:
        return pd.to_datetime(data['$date'], unit='ms')
    return data

# Convert all date fields
date_fields = ['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']
for field in date_fields:
    receipts[field] = receipts[field].apply(convert_dates)

# Normalize and join 'rewardsReceiptItemList'
if 'rewardsReceiptItemList' in receipts.columns:
    # Explode the 'rewardsReceiptItemList' to get one item per row
    exploded_items = receipts.explode('rewardsReceiptItemList')
    
    # Normalize the exploded items. Ensure to handle nulls if present to avoid errors.
    normalized_items = pd.json_normalize(exploded_items['rewardsReceiptItemList'].dropna())
                                    
    # Add a prefix to each column name in the normalized DataFrame
    normalized_items = normalized_items.add_prefix('rewardsReceiptItemList_')

    # Join exploded and normalized items with the main DataFrame
    receipts_expanded = exploded_items.drop('rewardsReceiptItemList', axis=1).reset_index(drop=True)
    receipts_final = pd.concat([receipts_expanded, normalized_items], axis=1)

# Display the modified DataFrame to check results
receipts_final

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,...,rewardsReceiptItemList_itemNumber,rewardsReceiptItemList_originalMetaBriteQuantityPurchased,rewardsReceiptItemList_pointsEarned,rewardsReceiptItemList_targetPrice,rewardsReceiptItemList_competitiveProduct,rewardsReceiptItemList_originalFinalPrice,rewardsReceiptItemList_originalMetaBriteItemPrice,rewardsReceiptItemList_deleted,rewardsReceiptItemList_priceAfterCoupon,rewardsReceiptItemList_metabriteCampaignId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31.000,2021-01-03 15:25:31.000,2021-01-03 15:25:31,2021-01-03 15:25:36.000,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43.000,2021-01-03 15:24:43.000,2021-01-03 15:24:43,2021-01-03 15:24:48.000,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43.000,2021-01-03 15:24:43.000,2021-01-03 15:24:43,2021-01-03 15:24:48.000,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37.000,2021-01-03 15:25:37.000,NaT,2021-01-03 15:25:42.000,NaT,5.0,2021-01-03 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34.000,2021-01-03 15:25:34.000,2021-01-03 15:25:34,2021-01-03 15:25:39.000,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,603d0b710a720fde1000042a,NaN,NaN,2021-03-01 15:42:41.873,2021-03-01 15:42:41.873,NaT,2021-03-01 15:42:41.873,NaT,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7377,603cf5290a720fde10000413,NaN,NaN,2021-03-01 14:07:37.664,2021-03-01 14:07:37.664,NaT,2021-03-01 14:07:37.664,NaT,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7378,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28.000,2021-03-01 13:07:28.000,NaT,2021-03-01 13:07:29.000,NaT,25.0,2020-08-17 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7379,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28.000,2021-03-01 13:07:28.000,NaT,2021-03-01 13:07:29.000,NaT,25.0,2020-08-17 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Missing Values

In [9]:
# Display the number of missing values in each column 
print(receipts_final.isnull().sum())

_id                                                             0
bonusPointsEarned                                            1401
bonusPointsEarnedReason                                      1401
createDate                                                      0
dateScanned                                                     0
finishedDate                                                 1411
modifyDate                                                      0
pointsAwardedDate                                            1301
pointsEarned                                                 1128
purchaseDate                                                  458
purchasedItemCount                                            484
rewardsReceiptStatus                                            0
totalSpent                                                    435
userId                                                          0
rewardsReceiptItemList_barcode                               4291
rewardsRec

There are huge amount of missing data and has to be treated and lot of unwated columns to be removed

### Duplicate Rows

In [10]:
# Check for duplicate rows
duplicate_rows = receipts_final.duplicated()
print(f"Number of duplicate rows: {duplicate_rows.sum()}")

Number of duplicate rows: 111


In [11]:
# Remove duplicate rows and keep the first occurrence
receipts_final = receipts_final.drop_duplicates()

# Check the DataFrame again to confirm duplicates are removed
print("Number of duplicate rows after removal:", receipts_final.duplicated().sum())


Number of duplicate rows after removal: 0


There were 111 duplicate rows and can affect data quality by leading to inaccuarate analysis hence it was removed.

### Data Consistency 

In [12]:
# Example: Check for any negative values in 'totalSpent'
print("Entries with negative totalSpent:", (receipts_final['totalSpent'] < 0).sum())

Entries with negative totalSpent: 0


To check if data follows specific format like in the above case where total spent should not be a negative value

# Users Data

In [13]:
# The actual Json file path is defined
file_path = 'C:/Users/ASUS/Downloads/Fetch/users.json'

# Load the JSON file into a DataFrame
users = pd.read_json(file_path, lines=True)

# Display the DataFrame
print(users)


                                      _id  active               createdDate  \
0    {'$oid': '5ff1e194b6a9d73a3a9f1052'}    True  {'$date': 1609687444800}   
1    {'$oid': '5ff1e194b6a9d73a3a9f1052'}    True  {'$date': 1609687444800}   
2    {'$oid': '5ff1e194b6a9d73a3a9f1052'}    True  {'$date': 1609687444800}   
3    {'$oid': '5ff1e1eacfcf6c399c274ae6'}    True  {'$date': 1609687530554}   
4    {'$oid': '5ff1e194b6a9d73a3a9f1052'}    True  {'$date': 1609687444800}   
..                                    ...     ...                       ...   
490  {'$oid': '54943462e4b07e684157a532'}    True  {'$date': 1418998882381}   
491  {'$oid': '54943462e4b07e684157a532'}    True  {'$date': 1418998882381}   
492  {'$oid': '54943462e4b07e684157a532'}    True  {'$date': 1418998882381}   
493  {'$oid': '54943462e4b07e684157a532'}    True  {'$date': 1418998882381}   
494  {'$oid': '54943462e4b07e684157a532'}    True  {'$date': 1418998882381}   

                    lastLogin         role signUpSo

In [14]:
# Flatten the '_id' field
users['_id'] = users['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) and '$oid' in x else x)

# Convert all date fields directly using a lambda function
date_fields = ['createdDate', 'lastLogin']
for field in date_fields:
    users[field] = users[field].apply(lambda x: pd.to_datetime(x['$date'], unit='ms') if isinstance(x, dict) and '$date' in x else x)

# Display the modified DataFrame to check results
users.head(5)

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554,2021-01-03 15:25:30.597,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858,consumer,Email,WI


### Missing Values 

In [15]:
# Display the number of missing values in each column 
print(users.isnull().sum())

_id              0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64


There are few missing values to be treated i.e. Last login could be null because no one really logged in so it can be null or can be replaced with last login. signupsource and state can be replaced with mode or unknown

### Duplicate Rows

In [16]:
# Check for duplicate rows
duplicate_rows = users.duplicated()
print(f"Number of duplicate rows: {duplicate_rows.sum()}")

Number of duplicate rows: 283


In [17]:
# Remove duplicate rows and keep the first occurrence
users = users.drop_duplicates()

# Check the DataFrame again to confirm duplicates are removed
print("Number of duplicate rows after removal:", users.duplicated().sum())

Number of duplicate rows after removal: 0


# Q2 : Write queries that directly answer predetermined questions from a business stakeholder 

In [18]:
from pandasql import sqldf

# Function to allow running SQL query directly on pandas DataFrame
pysqldf = lambda q: sqldf(q, globals())

sqldf function is used to run sql queries in python and I have used My SQL

### 1. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [19]:
receipts_final['rewardsReceiptStatus'].unique()

array(['FINISHED', 'REJECTED', 'FLAGGED', 'SUBMITTED', 'PENDING'],
      dtype=object)

In [20]:
query = """
WITH AverageSpends AS (
    SELECT 
        rewardsReceiptStatus,
        AVG(totalSpent) AS average_spend
    FROM 
        receipts_final
    WHERE 
        rewardsReceiptStatus IN ('Accepted', 'Rejected')
    GROUP BY 
        rewardsReceiptStatus
)

SELECT
    CASE 
        WHEN MAX(average_spend) FILTER (WHERE rewardsReceiptStatus = 'Accepted') >
             MAX(average_spend) FILTER (WHERE rewardsReceiptStatus = 'Rejected')
        THEN 'Accepted has a greater average spend'
        ELSE 'Rejected has a greater average spend'
    END AS Result
FROM 
    AverageSpends;
"""

result = pysqldf(query)
result

,Result
0,Rejected has a greater average spend


I first checked the unique receipt status to see if both the status i.e. accepted and rejected exists, since we don't have accepted status, we can conclude without querying that rejected has greater average spend for reciepts. I have also show the SQL approach in this case.

### 2. When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [21]:
query = """
WITH Totalitemspurchased AS (SELECT 
    rewardsReceiptStatus,
    SUM(purchasedItemCount) AS total_items_purchased
FROM 
    receipts_final
WHERE 
    rewardsReceiptStatus IN ('Accepted', 'Rejected')
GROUP BY 
    rewardsReceiptStatus)
    
SELECT
    CASE 
        WHEN MAX(total_items_purchased) FILTER (WHERE rewardsReceiptStatus = 'Accepted') >
             MAX(total_items_purchased) FILTER (WHERE rewardsReceiptStatus = 'Rejected')
        THEN 'Accepted has a greater items purchased in total'
        ELSE 'Rejected has a greater items purchased in total'
    END AS Result
FROM 
    Totalitemspurchased;
"""

# Execute the query
result = pysqldf(query)
result

,Result
0,Rejected has a greater items purchased in total


In this case also Rejected status has greater item purchased in total due to the absence of accepted status in the source

### Q3 : Which brand has the most spend among users who were created within the past 6 months?

In [22]:
#Extract unique dates(years) to identify if 2024 exists
unique_dates = users['createdDate'].dt.year.unique()
print("The unique dates in 'createdDate' is:", unique_dates)

#Identify the max or the latest date in the database
max_date = users['createdDate'].max()
print("The maximum date in 'createdDate' is:", max_date)

The unique dates in 'createdDate' is: [2021 2020 2015 2017 2014]
The maximum date in 'createdDate' is: 2021-02-12 14:11:06.240000


In [23]:
query = """
WITH RecentUsers AS (
    SELECT _id 
    FROM users 
    WHERE createdDate > date('2021-02-12', '-6 months')
),
TotalSpends AS (
    SELECT 
        r.rewardsReceiptItemList_brandCode,
        SUM(r.totalSpent) AS summedTotalSpent 
    FROM 
        receipts_final r 
    WHERE 
        r.userId IN (SELECT _id FROM RecentUsers)
        AND r.rewardsReceiptItemList_brandCode IS NOT NULL
    GROUP BY 
        r.rewardsReceiptItemList_brandCode
)
SELECT 
    rewardsReceiptItemList_brandCode,
    summedTotalSpent 
FROM 
    TotalSpends
ORDER BY 
    summedTotalSpent DESC 
LIMIT 1;

"""

# Execute the query
result = pysqldf(query)
result

,rewardsReceiptItemList_brandCode,summedTotalSpent
0,BEN AND JERRYS,198695.24


I currently don't have data for 2023 or 2024 to evaluate the brand that has spent the most among users in last 6 months. Hence, I identified the latest transaction date in dataset and analyzed brand expenditures for the six months preceding this date. The analysis revealed that 'Ben & Jerry's' was the top spending brand during this period.

# Q3. Evaluate Data Quality Issues in the Data Provided

I have performed all data quality checks above after reading the data and converting it to dataframe. Some of the data quality check are identifying missing values, Duplicates, data inconsistencies and have also given the solution on how to address these issues and some of them are addressed like duplicate rows removal.

The main issue that I observed was no relation between receipt items list with either reciepts or brands. There's no unique id or brand codes to join or match with.

In [24]:
#Count Unique Brand Codes
unique_receipts_brand_codes = len(set(receipts_final['rewardsReceiptItemList_brandCode']))
print("The uniques brand code in 'receipts is:", unique_receipts_brand_codes)

unique_brands_brand_codes = len(set(brands_final['brandCode']))
print("The unique brand code in brands is:", unique_brands_brand_codes)


The uniques brand code in 'receipts is: 228
The unique brand code in brands is: 898


In [25]:
# Using left join to find unmatched entries
merged_data = pd.merge(receipts_final, brands_final, how='left', left_on='rewardsReceiptItemList_brandCode', right_on='brandCode', indicator=True)

In [26]:
# Identify Missing Brand Codes
missing_brand_codes = merged_data[merged_data['_merge'] == 'left_only']['rewardsReceiptItemList_brandCode'].unique()
print("Brand codes in receipts with no match in brands table:", missing_brand_codes)

Brand codes in receipts with no match in brands table: ['MISSION' 'BRAND' 'KRAFT EASY CHEESE' 'WINGSTOP' 'GERM-X'
 'BEN AND JERRYS' 'BORDEN' 'KLARBRUNN' 'HY-VEE' 'LIGHT & FIT GREEK'
 "CONNIE'S PIZZA" "VAN DE KAMP'S" 'HATCH FARMS' "KELLOGG'S" 'TEMPTATIONS'
 "NATURE'S PATH ORGANIC" 'DOLE' 'EL MONTEREY' 'BIGELOW' 'HY-VEE SELECT'
 'KIKKOMAN' 'SPECIAL K' 'HILLSHIRE FARM' 'JUST BARE' "LAURA'S LEAN BEEF"
 'CAL-ORGANIC FARMS' "BUSH'S BEST" 'FOLGERS' 'KASHI' 'LIPTON'
 'GREEN GIANT' 'HARVEST SNAPS' "THAT'S SMART!" 'ADVIL'
 'CHICKEN OF THE SEA' 'STARKIST' 'TIC TAC' 'SO DELICIOUS' 'WONDERFUL'
 'LIGHT & FIT' 'HANOVER' 'HIDDEN VALLEY' 'DANNON' 'FAGE' 'ORAL-B GLIDE'
 "CAMPBELL'S" "FRENCH'S" 'CRISPIX' 'KING ARTHUR FLOUR' 'KITCHEN BASICS'
 'MCCORMICK' 'OLD EL PASO' 'ZESTA' 'AZTECA' 'BUNNY'
 'HONEY BUNCHES OF OATS' 'SIMPLE TRUTH ORGANIC' 'BOTA BOX' 'DARE' 'LINDT'
 'ORGANIC ROOT STIMULATOR' "MERKT'S" 'MORTON' 'FRONTERA' 'KARO' 'CHEESE'
 "FLORIDA'S NATURAL" 'BLUE DIAMOND' 'LUNDBERG FAMILY FARMS' 'NUTRI-GR

The analysis confirms that there are brand codes used in receipts that do not have a corresponding entry in the brands table. This discrepancy can lead to issues when linking these two datasets for comprehensive analysis, potentially impacting business insights and operational decisions when there are no foreign keys. I ahve shown in the data model on what attributes are required in receipts table to link with other tables.